In [ ]:

import dash
import core
from Scripts.actions import * 
from dash import html,dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate

# General Settings
# openai.api_type = "azure"
# openai.api_base = "https://chatbotv2.openai.azure.com/"
# openai.api_version = "2022-06-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

divIntro= html.Div(children=[
    dcc.ConfirmDialog(
        id='connect-dialog',
        message="Changing Chatbot Options",
    ),
    dcc.ConfirmDialog(
        id='message-dialog',
    ),
    dbc.Container(fluid=True,className="py-3",
    children=[
        html.H1("MarBot", className="display-3"),
        html.P("This Chatbot is a proof of concept that demonstrates the use of GPT-3 and lower versions as new technologies.\
                It connects to Seeking Alpha and searches for trending news or analysis. The website provides two options for users,\
                Open question' and 'Summary', which allows users to ask questions or create summaries about the articles they choose.",className="lead")
                ],
            )], className="p-3 bg-light rounded-3",
)

divOptions = html.Div(
    dbc.Container( className="display-7", style={'margin-left':'5px'},children=[
        html.H3("Please Select One Option", style={'margin': '10px'}),
        # Chat Option
        html.Div(
            style= {'display':'flex'},
            children=[
                html.H4("ChatBot Mode", style={'margin': '12px'}),
                dbc.RadioItems(id="chatbot-mode",style = {'margin':'15px'}, options=[
                    {"label": "Open Questions", "value": 'OpenChatbot'},
                    {"label": "Smart  Summary", "value": 'Summary'}], inline=True)
                    ]
            ),

        html.Div(style= {'display':'flex'},children=[
            html.H4("Category", style={'margin': '12px'}),
            dbc.RadioItems(id="selected-mode",style = {'margin':'15px'}, options=[
                {"label": "Latest New", "value": 'latest-articles'},
                {"label": "Market News", "value": 'market-news'}], inline=True)]),

            # Div For Select
            html.Div(style={'display':'flex'}, children = [
                html.H4("Select Article", style={'margin': '12px', 'width':'30%'}),
                 dbc.Select(id="dropdown-article")]),

            ]),

       
    className="p-3 bg-light rounded-3",
    style={'border':'1px solid black', 'display': 'inline-block', 'width': '100%', 'vertical-align': 'top', "margin-top":'5px'}
)

divChat = html.Div(id = 'response-container', children=[])

app.layout = html.Div(style={'width': '100%', 'display': 'block', 'padding':'5px'}, children=[
    divIntro,
    html.Hr(style={'margin':'0px'}),
    divOptions,
    divChat])


# Connect2SeekingAlpha
@app.callback(
    Output("connect-dialog", 'displayed'),
    [Input('chatbot-mode', 'value')],
    [State('dropdown-article', 'options')],
)
def DownloadingArticles(value, options):
    if value is None:
        raise PreventUpdate
    else:
        core.TMP_CONTAINER = Connect2Seeking(trigger=True)
        return True

# Adding Article Selection 
@app.callback(
    Output("dropdown-article", 'options'),
    [Input('selected-mode','value')]
)
def UpdateList(mode:str):
    if mode is None:
        raise PreventUpdate
    else:
        tmp_options = []
        for name, _ in core.TMP_CONTAINER[mode].items():
            tmp_options.append({'label': name, 'value': name})
        return tmp_options

# Activate Chat Summary 
@app.callback(
    Output('response-container', 'children'),
    [Input('chatbot-mode','value')]
)
def UpdatingInterface (value:str):
    if value is None:
        raise PreventUpdate
    elif value == core.OPTIONS_MODE[0]:
        interface = html.Div(style={'display':'block'},children=[
            # ChatBot Answer Interface
            html.Div(id='chatbot-response', style={'min-height':'100px','border':'1px solid black','border-radius':'5px','margin-top':'5px'}, children = [
                html.Div(id='article-show', style={'padding': '10px', 'font-size': '18px'})
            ]),

            # User Interface
            html.Div(id='user-response', style={'display':'flex', 'margin-top':'5px'}, children = [
                dbc.Input(id='message', placeholder='Type your message here', type='text', style={'margin-right':'5px'}),
                html.Button("Submit", id="submit-button", className="btn btn-primary ml-2", n_clicks=0)]
                )
            ]
        )
        return interface
    elif value == core.OPTIONS_MODE[1]:
        interface = html.Div(className='d-flex',style={'margin-top': '10px'}, children=[
            html.Div(id='Summary-container', style={'min-width':'92%','min-height': '100px','border':'1px solid black','border-radius':'5px','margin-right':'5px'}),
            html.Button("Generate Summary", id="summary-button", className="btn btn-primary ml-2", n_clicks=0)])
        return interface



@app.callback(
    [Output('article-show', 'children')],
    [Input('selected-mode','value'),
     Input('dropdown-article','value')]
)

def ShowPartialNews (category, article):
    if any(criteria is None for criteria in [category, article]):
        raise PreventUpdate
    else:
        tmp_link_article = core.TMP_CONTAINER[category][article]
        tmp_article = GetArticle(Option=category, Selected=article, return_article=True)
        return [f"Partial Article : {tmp_article[:1000]} ... {tmp_article[-500:]} \n\nArticle Link:{tmp_link_article}"]
        # return html.Div(f"Partial Article : {tmp_article}\nEntire Article :{tmp_link_article}",style={'padding': '10px', 'font-size': '18px'})


@app.callback(
    [
        Output('message-dialog', 'message'),
        Output('message-dialog', 'displayed'),
        Output('Summary-container','children'),
    ],
    [Input('selected-mode','value'),
    Input('dropdown-article','value'),
    Input('chatbot-mode','value'),
    Input('summary-button', 'n_clicks')
    ]
)
def CreateSummary (Option,Article,Mode,n_clicks):

    if (n_clicks != 0):
        if (n_clicks < 3):
            print(Option,Article,Mode,n_clicks)
            if (Option != None) and (Article != None) and (Mode != None):
                Message, Summary = UserQuestion(Option,Article,Mode, Question = None)
                return Message, True, html.Div(Summary,style={'padding': '10px', 'font-size': '18px'})
            else:
                return  "One or more Options are empty", True, html.Div("Please Make sure Category and Article are Selected",style={'padding': '10px', 'font-size': '18px'})
        else:
            return "Only 3 Summaries, No more Credits", True, html.Div("Only 3 Summaries",style={'padding': '10px', 'font-size': '18px'})
    else:
        raise PreventUpdate


    


if __name__ == '__main__':
    app.run_server(debug=False)

In [25]:
import dash
import core
from Scripts.actions import * 
from dash import html,dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Container Nav and Instruction
DivNav = html.Div(children=[

    # Container Nav
    html.Div(children=[
        dbc.NavbarSimple(dbc.NavItem(dbc.NavLink("More Info", href="https://www.linkedin.com/in/marvin-garcia-917127184/")),
        brand="MarBot By Marvin G", brand_href="https://www.linkedin.com/in/marvin-garcia-917127184/",color="dark",dark=True)
    ]),

    # Instructions
    html.Div(className="p-3 bg-light rounded-3", children=[
        html.P("This Chatbot is a proof-of-concept that utilizes GPT-3 and lower versions to showcase the capabilities of new technologies.\
            It connects to the financial news website Seeking Alpha and allows users to search for trending news and analysis.\
            The bot offers two options for users - the ability to ask open-ended questions or generate summaries of the articles they select.", style={'text-align':'center'}, className="lead"
        )
    ])
])

# Container Options 
DivOption = html.Div(id='div-options', className="p-3 bg-light rounded-3", style={'min-width':'30%','width':'35%'}, children=[

    # Main Accordion
    dbc.Accordion(style={'with':'40%'}, children=[
        
        # Latest-Trend
        dbc.AccordionItem(title = "Step 1. Categories" , style={'width':'100%'},children =[
            html.Div(style= {'display':'flex'},children=[
                html.H4("Options", style={'margin': '12px'}),
                dbc.RadioItems(id="selected-type",style = {'margin':'15px'}, options=[
                    {"label": "Latest New", "value": 'latest-articles'},
                    {"label": "Market News", "value": 'market-news'}], inline=True
                )
            ])
        ]),

        # Articles
        dbc.AccordionItem(title="Step 2. Articles", style={'width':'100%'},children=[
            html.Div(style={'display':'block'}, children = [
                html.H4("Select Article", style={'font-size':'1.2rm'}),
                dcc.Loading(id='spinner',type="dot", fullscreen =True, children= [dbc.Select(id="dropdown-article")])
            ]),
        ]),

        # Chat Mode
        dbc.AccordionItem(title = "Step 3. ChatBot Mode" , style={'width':'100%'}, children =[
            html.Div(style= {'display':'block'},children=[
                html.H4("MarBot Modes", style={'margin': '12px'}),
                dbc.RadioItems(id="chatbot-mode",style = {'margin':'15px'}, inline=True, options=[
                    {"label": "Open Questions", "value": 'OpenChatbot'},
                    {"label": "Smart  Summary", "value": 'Summary'}
                ])
            ])
        ]),
    ]),

    # Warning
    html.Div(style={'width':'100%'},children=[
            dbc.Card(style = {'margin-top':'10px'},className="card-text",children=[
                dbc.CardBody( style = {'font-size': '1.23rem','font-weight': '300'}, children=[
                    html.P(children=[
                        "If you're experiencing any issues with the website, try reloading the page.",
                        "Please note that this is just a proof of concept and there is a ",
                        html.Code('$50'),
                        " Credit limit for the OpenAI API.",
                        "Once the credit is used, the demo will be shut down."
                    ])
                ])
            ])
        ]),

    # Reset Button 
    html.Button("Reset", id="reset-button", className="btn btn-danger", style = {'margin-top':'10px','width':'100%'})

])

# Message Before Starting
DivChatSum = html.Div(style={'width':'100%'},children=[
    dbc.Card(style={'height':'100%'},className="card-text",children=[
        dbc.CardBody(id ="chatbot-response", children=[
            html.H5("Friendly Reminder", className="card-title"),
            html.P(children=[
                "Wait , The Chatbot will ",
                html.Code( "answer") ,
                " in here."
            ]),
        ])
    ])
])
        
# Middle Container
DivMid = html.Div(id='response', style={'display':'inline-flex','width':'100%'}, children=[
    DivOption,
    DivChatSum

])

@app.callback(
    [
        Output('spinner', 'children'),
        Input('selected-type', 'value')
    ]
)


def LoadingConnection (value):
    """ Load spinner"""
    if value is None:
        raise PreventUpdate
    else:
        return [dbc.Spinner(id="spinner-connection", spinner_style={"width": "3rem", "height": "3rem"}, delay_show =5, delay_hide=1, fullscreen=True)]

# Initiating Connection with SeekingAlpha 
@app.callback(
    [
        Output('dropdown-article', 'options'),
        Input('selected-type', 'value')
    ]
)    

# def DownloadingArticles(value):
#     """ Connect to SeekingAlpha
#     Parameters;
#         value: string, Latest new | market news """
#     if value is None:
#         raise PreventUpdate
#     else:
#         core.TMP_CONTAINER = Connect2Seeking(trigger=True)
#         return True

# Initiating Connection with SeekingAlpha 



    









app.layout = html.Div(children=[
    DivNav,
    DivMid])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving 

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2023 16:49:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 16:49:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 16:49:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 16:49:29] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 16:49:29] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [19/Jan/2023 16:49:30] "POST /_dash-update-component HTTP/1.1" 200 -


In [21]:
help(dbc.Spinner)

Help on class Spinner in module dash_bootstrap_components._components.Spinner:

class Spinner(dash.development.base_component.Component)
 |  Spinner(children=None, id=undefined, fullscreen_style=undefined, spinner_style=undefined, fullscreen_class_name=undefined, fullscreenClassName=undefined, spinner_class_name=undefined, spinnerClassName=undefined, color=undefined, type=undefined, size=undefined, fullscreen=undefined, delay_hide=undefined, delay_show=undefined, show_initially=undefined, **kwargs)
 |  
 |  A Spinner component.
 |  Render Bootstrap style loading spinners using only CSS.
 |  
 |  This component can be used standalone to render a loading spinner, or it can
 |  be used like `dash_core_components.Loading` by giving it children. In the
 |  latter case the chosen spinner will display while the children are loading.
 |  
 |  Keyword arguments:
 |  
 |  - children (a list of or a singular dash component, string or number; optional):
 |      The children of this component.
 |  